In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path(".").resolve() #/CS484/project/ or however yours looks
DATA_DIR = ROOT / "csv_files"
TRAIN_DIR = DATA_DIR / "train"

print("TRAIN_DIR:", TRAIN_DIR)

# ---- Config ----
BASE_FILE = TRAIN_DIR / "train_base.csv"

READ_CHUNK_SIZE = 200_000
CASE_ID = "case_id"

TRAIN_DIR: C:\Users\tahir\CS484\project\csv_files\train


In [2]:
#loading the data in a dataframe

df_base = pd.read_csv(BASE_FILE, low_memory=False)
#print(f"df base\n{df_base}")
df_base20 = df_base.head(20).copy()
base_ids = set(df_base20[CASE_ID].unique())

print(f"Loaded {len(df_base20)} base rows")
print(f"{len(base_ids)} unique IDs exist")


Loaded 20 base rows
20 unique IDs exist


In [3]:
def read_filtered(fp: Path, case_id: str, ids: set) -> pd.DataFrame:
    out = []
    try:
        for chunk in pd.read_csv(fp, chunksize=READ_CHUNK_SIZE, low_memory=False):
            if case_id not in chunk.columns:
                return pd.DataFrame()
            filt = chunk[chunk[case_id].isin(ids)]
            if not filt.empty:
                out.append(filt)
    except pd.errors.EmptyDataError:
        return pd.DataFrame()
    return pd.concat(out, ignore_index=True) if len(out)>0 else pd.DataFrame()


In [4]:
def collapse_rows_per_id(df: pd.DataFrame, case_id: str) -> pd.DataFrame:
    """
    For each case_id, sort by a detected date column if present; otherwise by original order.
    Then, for each column, take the last non-null value across that ID's rows.
    Returns one row per case_id with maximal filled values (no new engineered features).
    """
    df = df.copy()

    # 1) try to detect a date-like column to determine row recency
    dcol = None
    for c in df.columns:
        if c == case_id:
            continue
        if "date" in c.lower():
            parsed = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
            if parsed.notna().any():
                df[c] = parsed
                dcol = c
                break

    # 2) sort by id and recency (if we have a date col)
    if dcol is not None:
        df = df.sort_values([case_id, dcol])
    else:
        # preserve original order within each id
        df["_row_order__"] = np.arange(len(df))
        df = df.sort_values([case_id, "_row_order__"])

    # 3) within each id, forward-fill so the last row holds the latest non-null per column
    #    then take that last row
    def _pick_last_filled(g: pd.DataFrame) -> pd.Series:
        g_ff = g.ffill()  # forward-fill within the group, column-wise
        return g_ff.iloc[-1]  # last row now contains last non-null per column

    collapsed = (
        df.groupby(case_id, as_index=False, sort=False)
          .apply(_pick_last_filled)
          .reset_index(drop=True)
    )

    # cleanup helper column if we created it
    if "_row_order__" in collapsed.columns:
        collapsed = collapsed.drop(columns=["_row_order__"], errors="ignore")

    return collapsed


In [5]:
def integrate_file_into_train_all(train_all: pd.DataFrame, df_part: pd.DataFrame, case_id: str, overwrite: bool = False) -> pd.DataFrame:
    df_pick = collapse_rows_per_id(df_part, case_id)

    train_all[case_id] = pd.to_numeric(train_all[case_id], errors="coerce")
    df_pick[case_id] = pd.to_numeric(df_pick[case_id],   errors="coerce")

    # map by ID for fast column-wise fills
    by_id = df_pick.set_index(case_id)
    for col in by_id.columns:
        if col == case_id:
            continue
        if col not in train_all.columns:
            train_all[col] = np.nan
        mapped = train_all[case_id].map(by_id[col])
        if overwrite:
            train_all[col] = mapped.where(mapped.notna(), train_all[col])
        else:
            train_all[col] = train_all[col].fillna(mapped)
    return train_all



In [6]:
train_all = df_base20.copy()
for fp in sorted(TRAIN_DIR.glob("train_*.csv")):
    #just ignore train_base.csv
    if fp.name == "train_base.csv":
        continue

    print(f"Processing {fp.name} ...")
    df_part = read_filtered(fp, CASE_ID, set(train_all[CASE_ID]))
    if df_part.empty:
        print(f"{df_part} was skipped, no matching IDs or no ID column exist")
        continue

    train_all = integrate_file_into_train_all(train_all, df_part, CASE_ID, overwrite=False)
    print("integrated all files")

print("\n\nDone.")
print(f"train_all shape:, {train_all.shape}")

Processing train_applprev_1_0.csv ...


C:\Users\tahir\AppData\Local\Temp\ipykernel_33012\2138270767.py:15: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
C:\Users\tahir\AppData\Local\Temp\ipykernel_33012\2138270767.py:32: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  g_ff = g.ffill()  # forward-fill within the group, column-wise
C:\Users\tahir\AppData\Local\Temp\ipykernel_33012\2138270767.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future vers

integrated all files
Processing train_applprev_1_1.csv ...
Empty DataFrame
Columns: []
Index: [] was skipped, no matching IDs or no ID column exist
Processing train_applprev_2.csv ...


C:\Users\tahir\AppData\Local\Temp\ipykernel_33012\2138270767.py:32: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  g_ff = g.ffill()  # forward-fill within the group, column-wise
C:\Users\tahir\AppData\Local\Temp\ipykernel_33012\2138270767.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_pick_last_filled)


integrated all files
Processing train_credit_bureau_a_1_0.csv ...
Empty DataFrame
Columns: []
Index: [] was skipped, no matching IDs or no ID column exist
Processing train_credit_bureau_a_1_1.csv ...
Empty DataFrame
Columns: []
Index: [] was skipped, no matching IDs or no ID column exist
Processing train_credit_bureau_a_1_2.csv ...
Empty DataFrame
Columns: []
Index: [] was skipped, no matching IDs or no ID column exist
Processing train_credit_bureau_a_1_3.csv ...
Empty DataFrame
Columns: []
Index: [] was skipped, no matching IDs or no ID column exist
Processing train_credit_bureau_a_2_0.csv ...
Empty DataFrame
Columns: []
Index: [] was skipped, no matching IDs or no ID column exist
Processing train_credit_bureau_a_2_1.csv ...
Empty DataFrame
Columns: []
Index: [] was skipped, no matching IDs or no ID column exist
Processing train_credit_bureau_a_2_10.csv ...
Empty DataFrame
Columns: []
Index: [] was skipped, no matching IDs or no ID column exist
Processing train_credit_bureau_a_2_2.csv

...

integrated all files
Processing train_person_2.csv ...


C:\Users\tahir\AppData\Local\Temp\ipykernel_33012\2138270767.py:32: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  g_ff = g.ffill()  # forward-fill within the group, column-wise
C:\Users\tahir\AppData\Local\Temp\ipykernel_33012\2138270767.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_pick_last_filled)


integrated all files
Processing train_static_0_0.csv ...


C:\Users\tahir\AppData\Local\Temp\ipykernel_33012\2138270767.py:15: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
C:\Users\tahir\AppData\Local\Temp\ipykernel_33012\2138270767.py:15: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
C:\Users\tahir\AppData\Local\Temp\ipykernel_33012\2138270767.py:15: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict 

integrated all files
Processing train_static_0_1.csv ...
Empty DataFrame
Columns: []
Index: [] was skipped, no matching IDs or no ID column exist
Processing train_static_cb_0.csv ...
Empty DataFrame
Columns: []
Index: [] was skipped, no matching IDs or no ID column exist
Processing train_tax_registry_a_1.csv ...
Empty DataFrame
Columns: []
Index: [] was skipped, no matching IDs or no ID column exist
Processing train_tax_registry_b_1.csv ...
Empty DataFrame
Columns: []
Index: [] was skipped, no matching IDs or no ID column exist
Processing train_tax_registry_c_1.csv ...
Empty DataFrame
Columns: []
Index: [] was skipped, no matching IDs or no ID column exist


Done.
train_all shape:, (20, 259)


In [7]:
from IPython.display import display
import pandas as pd

pd.set_option("display.max_rows", None)       # show all 20 rows
pd.set_option("display.max_columns", None)    # show all columns
pd.set_option("display.width", None)

print(f"\nDone. train_all shape: {train_all.shape}")
display(train_all)



Done. train_all shape: (20, 259)


,case_id,date_decision,MONTH,WEEK_NUM,target,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,childnum_21L,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_status_367L,credacc_transactions_402L,credamount_590A,credtype_587L,currdebt_94A,dateactivated_425D,district_544M,downpmt_134A,dtlastpmt_581D,dtlastpmtallstes_3545839D,education_1138M,employedfrom_700D,familystate_726L,firstnonzeroinstldate_307D,inittransactioncode_279L,isbidproduct_390L,isdebitcard_527L,mainoccupationinc_437A,maxdpdtolerance_577P,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L,cacccardblochreas_147M,conts_type_509L,credacc_cards_status_52L,num_group2,birth_259D,birthdate_87D,childnum_185L,contaddr_district_15M,contaddr_matchlist_1032L,contaddr_smempladdr_334L,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,empl_employedtotal_800L,empl_industry_691L,empladdr_district_926M,empladdr_zipcode_114M,familystate_447L,gender_992L,housetype_905L,housingtype_772L,incometype_1044T,isreference_387L,language1_981M,mainoccupationinc_384A,maritalst_703L,personindex_1023L,persontype_1072L,persontype_792L,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,remitter_829L,role_1084L,role_993L,safeguarantyflag_411L,sex_738L,type_25L,addres_district_368M,addres_role_871L,addres_zip_823M,conts_role_79M,empls_economicalst_849M,empls_employedfrom_796D,empls_employer_name_740M,relatedpersons_role_762T,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,credamount_770A,credtype_322L,currdebt_22A,currdebtcredtyperange_828A,datefirstoffer_1144D,datelastinstal40dpd_247D,datelastunpaid_3546854D,daysoverduetolerancedd_3976961L,deferredmnthsnum_166L,disbursedcredamount_1113A,disbursementtype_67L,downpmt_116A,dtlastpmtallstes_4499206D,eir_270L,equalitydataagreement_891L,equalityempfrom_62L,firstclxcampaign_1125D,firstdatedue_489D,homephncnt_628L,inittransactionamount_650A,inittransactioncode_186L,interestrate_311L,interestrategrace_34L,isbidproduct_1095L,isbidproductrequest_292L,isdebitcard_729L,lastactivateddate_801D,lastapplicationdate_877D,lastapprcommoditycat_1041M,lastapprcommoditytypec_5251766M,lastapprcredamount_781A,lastapprdate_640D,lastcancelreason_561M,lastdelinqdate_224D,lastdependentsnum_448L,lastotherinc_902A,lastotherlnsexpense_631A,lastrejectcommoditycat_161M,lastrejectcommodtypec_5251769M,lastrejectcredamount_222A,lastrejectdate_50D,lastrejectreason_759M,lastrejectreasonclient_4145040M,lastrepayingdate_696D,lastst_736L,maininc_215A,mastercontrelectronic_519L,mastercontrexist_109L,maxannuity_159A,maxannuity_4075009A,maxdbddpdlast1m_3658939P,maxdbddpdtollast12m_3658940P,maxdbddpdtollast6m_4187119P,maxdebt4_972A,maxdpdfrom6mto36m_3546853P,maxdpdinstldate_3546855D,maxdpdinstlnum_3546846P,maxdpdlast12m_727P,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdlast6m_474P,maxdpdlast9m_1059P,maxdpdtolerance_374P,maxinstallast24m_3658928A,maxlnamtstart6m_4525199A,maxoutstandbalancel12m_4187113A,maxpmtlast3m_4525190A,mindbddpdlast24m_3658935P

In [9]:
cont = train_all["target"]
cont.head()

0    0
1    0
2    0
3    0
4    1
Name: target, dtype: int64

In [ ]:
print("Base IDs:", df_base.head(20)[CASE_ID].tolist())
print("Train_all IDs:", train_all[CASE_ID].tolist())
